In [9]:
# !pip install boostsa

In [1]:
from boostsa import Bootstrap
import pickle
import pandas as pd
import numpy as np
import itertools
import os
import math
import shutil
boot = Bootstrap()

In [2]:
home_dir = '/g100/home/userexternal/mhabibi0/'
work_dir = '/g100_work/IscrC_mental'

hdata_dir = os.path.join(home_dir, 'Data')
wdata_dir = os.path.join(work_dir, 'data')
uc_dir = os.path.join(wdata_dir, 'user_classification')
mit_dir = os.path.join(uc_dir,  'trained_models', 'it')

In [3]:
# read test data
path  = os.path.join(uc_dir, 'data_for_models_test.pkl')
df_gold = pd.read_pickle(path)
df_gold['male'] = df_gold['is_male'].astype(int)

# Discretize the 'age' column into four classes
age_intervals = [0, 19, 30, 40, 100]
age_labels = [0, 1, 2, 3]
df_gold = df_gold[df_gold['age']<=99]
df_gold['age_class'] = pd.cut(df_gold['age'], bins=age_intervals, labels=age_labels, right=False).astype(int)


df_gold = df_gold[['user_id', 'male','age_class']]

df_gold['user_id'] = df_gold['user_id'].astype(float)

In [40]:
# xlm

# bio+tweet gender
path = os.path.join(mit_dir, 'xlm_bio_tweet_test_gender.csv')
df_xlm_btg = pd.read_csv(path)


# bio+tweet age
path = os.path.join(mit_dir, 'xlm_bio_tweet_test_age.csv')
df_xlm_bta = pd.read_csv(path)

# bio only gender
path = os.path.join(mit_dir, 'xlm_bio_only_test_gender.csv')
df_xlm_bog = pd.read_csv(path)

# bio+tweet gender
path = os.path.join(mit_dir, 'xlm_bio_only_test_age.csv')
df_xlm_boa = pd.read_csv(path)


# reanme columns
list_df_xlms = [df_xlm_btg, df_xlm_bta, df_xlm_bog, df_xlm_boa]
for df_xlm in list_df_xlms:
    df_xlm.columns = ['user_id', 'prediction']

# M3: bio image
path = os.path.join(mit_dir, 'm3_bio_image_test_age_gender.csv')
df_m3_bi = pd.read_csv(path)
df_m3_bi['user_id'] = df_m3_bi['user_id'].astype(float)
df_m3_bi['pred_male'] = df_m3_bi['pred_male'].astype(int)
df_m3_bi['pred_age_class'] = df_m3_bi['pred_age_class'].astype(int)

df_m3_big = df_m3_bi[['user_id', 'pred_male']]
df_m3_big.columns = ['user_id', 'prediction']

df_m3_bia = df_m3_bi[['user_id', 'pred_age_class']]
df_m3_bia.columns = ['user_id', 'prediction']



# M3 bio only
path = os.path.join(mit_dir ,'m3_test_age_gender.csv')
df_m3_bo = pd.read_csv(path)

df_m3_bo['user_id'] = df_m3_bo['user_id'].astype(float)
df_m3_bo['pred_male'] = df_m3_bo['pred_male'].astype(int)
df_m3_bo['pred_age_class'] = df_m3_bo['pred_age_class'].astype(int)

df_m3_bog = df_m3_bo[['user_id', 'pred_male']]
df_m3_bog.columns = ['user_id', 'prediction']

df_m3_boa = df_m3_bo[['user_id', 'pred_age_class']]
df_m3_boa.columns = ['user_id', 'prediction']

In [5]:
# flan
path = '/g100_work/IscrC_mental/data/user_classification/trained_models/it/'

# FLAN-T5 bio gender
file = 'flan_bio_only_test_gender.csv'
df_flan_bog = pd.read_csv(path+file)

# flan bio only age
file = 'flan_bio_only_test_age.csv'
df_flan_boa = pd.read_csv(path+file)

# FLAN-T5 bio + tweets
# gender
file = 'flan_bio_tweets_test_gender.csv'
df_flan_btg = pd.read_csv(path+file)

# age
file = 'flan_bio_tweets_test_age.csv'
df_flan_bta =  pd.read_csv(path+file)

# GPT 3.5 bio

# age
file = 'gpt35_bio_test_age.csv'
df_gpt_boa = pd.read_csv(path+file)

# gender
file = 'gpt35_bio_test_gender.csv'
df_gpt_bog = pd.read_csv(path+file)


# GPT 3.5 bio + tweets
# age
file =  'gpt35_bio_tweets_test_age.csv'
df_gpt_bta = pd.read_csv(path+file)

# gender
file = 'gpt35_bio_tweets_test_gender.csv'
df_gpt_btg = pd.read_csv(path+file)

# transform gender dfs
dfs_g = [df_flan_bog, df_flan_btg, df_gpt_bog, df_gpt_btg]

for df_ in dfs_g:
    df_['pred_male'] = df_['prediction'].apply(lambda x: 1 if x == 'male' else 0)
    df_.drop(columns=[col for col in df_.columns if col not in ['user_id', 'pred_male']], inplace=True)
    df_.rename(columns={'pred_male': 'prediction'}, inplace=True)
    
    
# transform age dfs
dfs_a = [df_flan_boa, df_flan_bta, df_gpt_boa, df_gpt_bta]

for df_ in dfs_a:
    df_.drop(columns=[col for col in df_.columns if col not in ['user_id', 'prediction']], inplace=True)

In [10]:
# CV 
file = 'cv_test.pkl'
df_cv = pd.read_pickle(path+file)

df_cv['user_id'] = df_cv['user_id'].astype(float)

# gender
df_cv['pred_male'] = 1- df_cv['pred_gender_label_id']
df_cvg = df_cv[['user_id', 'pred_male']]
df_cvg.columns = ['user_id', 'prediction']

# age
df_cva = df_cv[['user_id','pred_age_label_id']]
df_cva.columns = ['user_id', 'prediction']

In [11]:
list_gender_dfs = [df_xlm_btg, df_xlm_bog, df_m3_big, df_m3_bog, df_flan_bog, 
                   df_flan_btg, df_gpt_bog, df_gpt_btg, df_cvg]

gender_df_names = ['xlm_btg', 'xlm_bog', 'm3_big', 'm3_bog', 'flan_bog',
                   'flan_btg', 'gpt_bog', 'gpt_btg', 'cvg']


list_age_dfs = [df_xlm_bta, df_xlm_boa, df_m3_bia, df_m3_boa, df_flan_boa,
                df_flan_bta, df_gpt_boa, df_gpt_bta, df_cva]

age_df_names = ['xlm_bta', 'xlm_boa', 'm3_bia', 'm3_boa', 'flan_boa',
                'flan_bta', 'gpt_boa', 'gpt_bta', 'cva']


In [54]:
def model_vs_m3_gender(df_gld, df_m3, df_mod):
    
    """
    Evaluate model performance agains M3
    """
    # merge gold labels with m3
    df_merged = df_gld.merge(df_m3, on='user_id', how='inner')
    df_merged = df_merged.rename(columns={'prediction':'m3_pred'})
    df_merged = df_merged.merge(df_mod, on='user_id', how='inner')
    
    
    print(f'len df merged: {len(df_merged)}')
    
    gold_labels = df_merged['male'].values.tolist()
    m3_preds= df_merged['m3_pred'].values.tolist()
    mod_preds = df_merged['prediction'].values.tolist()
    boot.test(targs=gold_labels, h0_preds = m3_preds ,h1_preds=mod_preds, n_loops=1000, sample_size=.5, verbose=True)
    

In [55]:
model_vs_m3_gender(df_gold[['user_id','male']], df_m3_big, df_xlm_btg)

len df merged: 1087
data shape:  (1087, 1)
sample size: 543
h0: h0 - h1: h1
targs count:    ['class 0 freq 388 perc 35.69%', 'class 1 freq 699 perc 64.31%']
h0 preds count: ['class 0 freq 251 perc 23.09%', 'class 1 freq 836 perc 76.91%']
h1 preds count: ['class 0 freq 362 perc 33.30%', 'class 1 freq 725 perc 66.70%']
F-measure...... - h0: 0.8105  - h1: 0.8718  - diff: 0.0613
precision...... - h0: 0.8740  - h1: 0.8785  - diff: 0.0045
recall......... - h0: 0.7894  - h1: 0.8663  - diff: 0.0769
accuracy....... - h0: 0.8427  - h1: 0.8841  - diff: 0.0414


bootstrap: 100%|███████████████████████████| 1000/1000 [00:08<00:00, 112.92it/s]


count sample diff f1   is twice tot diff f1....... 5    / 1000    p < 0.005  **
count sample diff prec is twice tot diff prec..... 412  / 1000    p < 0.412  
count sample diff rec  is twice tot diff rec ..... 0    / 1000    p < 0.0    **
count sample diff acc  is twice tot diff acc...... 23   / 1000    p < 0.023  *


In [56]:
def model_vs_m3_age(df_gld, df_m3, df_mod):
    
    """
    Evaluate model performance agains M3
    """
    # merge gold labels with m3
    df_merged = df_gld.merge(df_m3, on='user_id', how='inner')
    df_merged = df_merged.rename(columns={'prediction':'m3_pred'})
    df_merged = df_merged.merge(df_mod, on='user_id', how='inner')
    
    print(f'len df merged: {len(df_merged)}')
    
    gold_labels = df_merged['age_class'].values.tolist()
    m3_preds= df_merged['m3_pred'].values.tolist()
    mod_preds = df_merged['prediction'].values.tolist()
    boot.test(targs=gold_labels, h0_preds = m3_preds ,h1_preds=mod_preds, n_loops=1000, sample_size=.5, verbose=True)
    

In [57]:
model_vs_m3_age(df_gold[['user_id','age_class']], df_m3_bia, df_xlm_bta)

len df merged: 1087
data shape:  (1087, 1)
sample size: 543
h0: h0 - h1: h1
targs count:    ['class 0 freq 88 perc 8.10%', 'class 1 freq 231 perc 21.25%', 'class 2 freq 214 perc 19.69%', 'class 3 freq 554 perc 50.97%']
h0 preds count: ['class 0 freq 123 perc 11.32%', 'class 1 freq 140 perc 12.88%', 'class 2 freq 171 perc 15.73%', 'class 3 freq 653 perc 60.07%']
h1 preds count: ['class 0 freq 64 perc 5.89%', 'class 1 freq 238 perc 21.90%', 'class 2 freq 217 perc 19.96%', 'class 3 freq 568 perc 52.25%']
F-measure...... - h0: 0.4171  - h1: 0.6008  - diff: 0.1837
precision...... - h0: 0.4307  - h1: 0.6252  - diff: 0.1945
recall......... - h0: 0.4223  - h1: 0.5846  - diff: 0.1623
accuracy....... - h0: 0.5363  - h1: 0.6615  - diff: 0.1252


bootstrap: 100%|███████████████████████████| 1000/1000 [00:08<00:00, 114.42it/s]


count sample diff f1   is twice tot diff f1....... 0    / 1000    p < 0.0    **
count sample diff prec is twice tot diff prec..... 0    / 1000    p < 0.0    **
count sample diff rec  is twice tot diff rec ..... 0    / 1000    p < 0.0    **
count sample diff acc  is twice tot diff acc...... 0    / 1000    p < 0.0    **


### XLM + M3 vs M3-base

In [58]:
path = os.path.join(mit_dir, 'xlm_m3_untrained_agg_test.pickle')

df_xlm_m3= pd.read_pickle(path)

In [59]:
# gender comprasion with baseline m3
df_xlm_m3['male_f1_agg'] = 1 - df_xlm_m3['gender_f1_agg']

df_xlm_m3_g = df_xlm_m3[['user_id', 'male_f1_agg']]

df_xlm_m3_g.columns =  ['user_id', 'prediction']

model_vs_m3_gender(df_gold[['user_id','male']], df_m3_big, df_xlm_m3_g)

len df merged: 1119
data shape:  (1119, 1)
sample size: 559
h0: h0 - h1: h1
targs count:    ['class 0 freq 399 perc 35.66%', 'class 1 freq 720 perc 64.34%']
h0 preds count: ['class 0 freq 256 perc 22.88%', 'class 1 freq 863 perc 77.12%']
h1 preds count: ['class 0 freq 342 perc 30.56%', 'class 1 freq 777 perc 69.44%']
F-measure...... - h0: 0.8090  - h1: 0.9082  - diff: 0.0992
precision...... - h0: 0.8741  - h1: 0.9275  - diff: 0.0534
recall......... - h0: 0.7877  - h1: 0.8955  - diff: 0.1078
accuracy....... - h0: 0.8418  - h1: 0.9187  - diff: 0.0769


bootstrap: 100%|███████████████████████████| 1000/1000 [00:08<00:00, 112.57it/s]


count sample diff f1   is twice tot diff f1....... 0    / 1000    p < 0.0    **
count sample diff prec is twice tot diff prec..... 0    / 1000    p < 0.0    **
count sample diff rec  is twice tot diff rec ..... 0    / 1000    p < 0.0    **
count sample diff acc  is twice tot diff acc...... 0    / 1000    p < 0.0    **


In [60]:
# age
df_xlm_m3_a = df_xlm_m3[['user_id', 'age_f1_agg']]
df_xlm_m3_a.columns=  ['user_id', 'prediction']

In [61]:
model_vs_m3_age(df_gold[['user_id','age_class']], df_m3_bia, df_xlm_bta)

len df merged: 1087
data shape:  (1087, 1)
sample size: 543
h0: h0 - h1: h1
targs count:    ['class 0 freq 88 perc 8.10%', 'class 1 freq 231 perc 21.25%', 'class 2 freq 214 perc 19.69%', 'class 3 freq 554 perc 50.97%']
h0 preds count: ['class 0 freq 123 perc 11.32%', 'class 1 freq 140 perc 12.88%', 'class 2 freq 171 perc 15.73%', 'class 3 freq 653 perc 60.07%']
h1 preds count: ['class 0 freq 64 perc 5.89%', 'class 1 freq 238 perc 21.90%', 'class 2 freq 217 perc 19.96%', 'class 3 freq 568 perc 52.25%']
F-measure...... - h0: 0.4171  - h1: 0.6008  - diff: 0.1837
precision...... - h0: 0.4307  - h1: 0.6252  - diff: 0.1945
recall......... - h0: 0.4223  - h1: 0.5846  - diff: 0.1623
accuracy....... - h0: 0.5363  - h1: 0.6615  - diff: 0.1252


bootstrap: 100%|███████████████████████████| 1000/1000 [00:08<00:00, 114.61it/s]


count sample diff f1   is twice tot diff f1....... 0    / 1000    p < 0.0    **
count sample diff prec is twice tot diff prec..... 0    / 1000    p < 0.0    **
count sample diff rec  is twice tot diff rec ..... 0    / 1000    p < 0.0    **
count sample diff acc  is twice tot diff acc...... 0    / 1000    p < 0.0    **
